# Integrazione C-Python

In [33]:
import sys,os
import numpy as np

Come detto diverse volte nel corso delle lezioni passate le principali librerie scientifiche in python (*numpy*, *scipy*, *pandas* e altre) forniscono spesso un interfaccia python a funzioni scritte in lingiaggi compilati come C e Fortran per velocizzare e rendere più efficiente la parte di calcolo vero e proprio. 

Un altri motivo per interfacciare C/C++ e python potrebbe essere quell odi utlizzare librerie sftware già esistenti. 

Esitono diversi modi per interfacciare python con C o C++. Per una decrizione delle possibili scelte vedere ad esempio  https://scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html.

## Librerie Condivise 

Uno dei metodi più usati per per permettere di utilizzare il codice (C/C++) sviluppato all'interno di altri programmi è quello di creare librerie condivise.

Le librerie condivise sono librerie che vengono caricate all'avvio dei programmi. Una volta che una libreria condivisa è stata correttamente installata, tutti i programmi successivamente eseguiti ne faranno automaticamente uso.

In Linux e nei sistemi basati su UNIX in generale le librerie condvise hanno l'estenzione `.so` (Shared Object) mentre nei sistemi Windows ci si riferisce a file `.dll` (Dynamic-Link Libraries).

ATTENZIONE il funzionamento delle librerire `.so` e `.dll` non è easattamente identico, le informazioni di seguito fanno espresso riferimento a sistemi Linux e quindi a librerie `.so`.

Ogni libreria condivisa ha uno speciale nome chiamato *soname*. Il *soname§ è caratterizzato dal prefisso *lib*, dal nome della libreria, dall'estensione  `.so`, ad esempio il file della libreria chiamata *pippo*  sarà chaiamto 
`libpippo.so`.



#### LD_LIBRARY_PATH

la variabile ambientale LD_LIBRARY_PATH  definisce i percorsi dove cercare i filemper le librerie condivise, in amiera analoga alla variabile PYTHONPATH che definisce i percorsi per i moduli python.

## ctypes

Uno degli esempi più semplici da di modulo da utilizzare per creare un interfaccia python a del codice scritto in C è `ctypes`.  

`ctypes` non richiede istallazioni aggiuntive ed è abbastanza samplice da utilizzare, fornisce però un support limitato al C++.

La procedura per utlizzare codice C attraverso `ctypes`  può essere riassunta nei seguenti passi:

* scrivere codice C con funzioni che forniscano le funzionalità necessarie
* compilare il codice C per ottenere una libreria condivisa
* scrivere un modulo python che definisca l'interfaccia python alla libreria condivisa attarverso `ctypes` nel seguente modo:
    * importando la libreria condivisa
    * defienendo la mappatura deii tipi di dati di input/output fra C e python
    * definendo funzioni python che richaimino appropriatamente le funzioni definite in C.

### Il modulo somme

Per maggiore chairezza vediamo un esempio dimostrativo.

Definiamo  un modulo `somme` che deve fornire le sehuenti funzioanlità:
* somma dei primi *n* numeri naturali
* somma delle radici dei primi *n* numeri naturali
* somma degli elementi di un array

#### somme.c

Iniziamo implementando il codice C che esegua le suddette operazioni all'interno del file `somme.c`

```
#include <stdio.h>
#include <math.h>


// Somma primi n numeri naturali
int sum_n(int n){

  int somma = 0;

  int i;
  for( i=0; i<=n; ++i) 
    somma += i;

  return somma; 
}



// Somma delle radici quadrate dei primi n numeri naturali
double sum_sqrtn(int n){

  double somma = 0;

  int i;
  for( i=0; i<=n; ++i) 
    somma += sqrt(i);

  return somma; 
}



// Somma degli elementi di un array
// *av: puntatore ad un array di double
// n  : numero di elementi dell'array
double sum_array(double *av, int n){

  double somma = 0;
  int i; 
  
  for( i=0; i < n; ++i) 
    somma += av[i];

  return somma;
}
```

#### libsomme.so

La creazione di librerie condivise, nel cas di pacchett software strutturati, può essere un'operazione complessa che richied un Makefile; nel nostro caso è sufficiente utilizzare direttaemnet il compilatore `gcc` con il seguente comando da terminale:

```
gcc -o libsomme.so -shared somme.c

```

dove l'opzione `-o` fornisce il nome del file di output, e `-shared` specifica che l'output che si vuole è una libreria condivisa. 

Esaminando la cartella, in assenza di errori, dovrebbe essere presente il file `libsomme.so`

#### somme.py

Il file `somme.py`, definirà il modulo python che permette l'utilizzo delle funzioni C definite in `somme.c`

```
import numpy
import ctypes

# Carico la lireria libsomme (libsomme.so) che è presente nella cartella di lavoro  ('.')
_libsomme = numpy.ctypeslib.load_library('libsomme', '.')

# definizoine tipi di input (argtypes) e di output (restypes) per la funzione sum_n di libsomme 
_libsomme.sum_n.argtypes = [ctypes.c_int]
_libsomme.sum_n.restype  = ctypes.c_int

# definizoine tipi di input (argtypes) e di output (restypes) per la funzione sum_sqrtn di libsomme 
_libsomme.sum_sqrtn.argtypes = [ctypes.c_int]
_libsomme.sum_sqrtn.restype  = ctypes.c_double


# definizoine tipi di input (argtypes) e di output (restypes) per la funzione sum_array di libsomme 
_libsomme.sum_array.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), ctypes.c_int]
_libsomme.sum_array.restype  = ctypes.c_double


# utilizzo di _libsomme.sum_n
# il parametro n va necessariamente convertito in int
def sum_n(n):
    return _libsomme.sum_n(int(n))


# utilizzo di _libsomme.sum_sqrtn
# il parametro n va necessariamente convertito in int
def sum_sqrtn(n):
    return _libsomme.sum_sqrtn(int(n))


# utilizzo di _libsomme.sum_array
# il parametro n va necessariamente ricavato dall'array di input e convertito in int
# l'oggetto av va necessariamente convertito in array (potrebbbe essere anche  uno scalare, una lista, o una ntupla )
def sum_array(av):
    n = len(av)
    av = numpy.asarray(av, dtype=numpy.float)
    return _libsomme.sum_array(av, int(n))

```

#### Utilizzo modulo somme

Proviamo ad utilizzare il modulo `somme` derivato da `libsomme`. 

Per prima cosa bisogna includere la cartella del modulo nel PYTHONPATH

In [4]:
# Aggiungo la cartella del modul oal path python 
# in alternativa si può settare la variabile ambientale da terminale
#   export PYTHONTAH=$PYTHONPATH:/percorso/modulo
sys.path.append('../accessori/L10')

In [5]:
sys.path

['/home/sg/Documents/Didattica/MetodiComputazionali/metodi-computazionali-fisica/notebooks',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/sg/.local/lib/python3.8/site-packages',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '../accessori/L10']

In [22]:
import somme

In [23]:
help(somme)

Help on module somme:

NAME
    somme

FUNCTIONS
    sum_array(av)
        # utilizzo di _libsomme.sum_array
        # il parametro n va necessariamente ricavato dall'array di input e convertito in int
        # l'oggetto av va necessariamente convertito in array (potrebbbe essere anche  uno scalare, una lista, o una ntupla )
    
    sum_n(n)
        # utilizzo di _libsomme.sum_n
        # il parametro n va necessariamente convertito in int
    
    sum_sqrtn(n)
        # utilizzo di _libsomme.sum_sqrtn
        # il parametro n va necessariamente convertito in int

DATA
    __warningregistry__ = {'version': 979}

FILE
    /home/sg/Documents/Didattica/MetodiComputazionali/metodi-computazionali-fisica/accessori/L10/somme.py




Utilizziamo le funzoni del modulo `somme`

In [25]:
# sum_n
print(somme.sum_n(100))

5050


In [26]:
# sum_sqrtn
print(somme.sum_sqrtn(100))

671.4629471031477


In [27]:
# sum_array

aa = np.arange(20)
print(aa)
print(somme.sum_array(aa))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
190.0


#### Confronto con numpy e codice python interamente interpretato

In [29]:
# funzone python che fa la somma degl ielemnti di un array 
def mysum_array(av):
    s = 0
    for v in av:
        s += v
    return s

In [34]:
import time

In [50]:
# array di 10^6 elementi
along = np.arange(1e8)/3.33



In [51]:
# test funzione interpretatata
tstart_py = time.time()
res_py    = mysum_array(along)
tstop_py  = time.time()

In [52]:
# test funzione da modulo somme (ctypes)
tstart_c = time.time()
res_c    = somme.sum_array(along)
tstop_c  = time.time()

In [53]:
# test funzione sum numpy
tstart_np = time.time()
res_np    = along.sum()
tstop_np  = time.time()

In [55]:
# Confronto risultati

print('py  somma = {:.0f}   durata = {:.6f} s'.format( res_py, tstop_py-tstart_py))
print('C   somma = {:.0f}   durata = {:.6f} s'.format( res_c,  tstop_c -tstart_c))
print('np  somma = {:.0f}   durata = {:.6f} s'.format( res_np, tstop_np-tstart_np))

py  somma = 1501501486486486   durata = 5.230483 s
C   somma = 1501501486486486   durata = 0.219421 s
np  somma = 1501501486486486   durata = 0.049359 s
